# Multi-Stage Minimum Failure Example (MFE) Time Stepping Simulation

This notebook implements a multi-stage Runge-Kutta-like time integration scheme for solving a PDE system using Devito.

## Overview

To apply multi-stage methods to a PDE system, we first have to formulated as a first-order in time system:


$\frac{d}{dt}\boldsymbol{U}(\boldsymbol{x},t)=\boldsymbol{HU}(\boldsymbol{x},t)+\boldsymbol{f}(\boldsymbol{x},t),\hspace{3cm}(1)$

where $\boldsymbol{x}$ is a vector with the spatial variables (x in 1D, (x,y) in 2D, (x,y,z in 3D)), $\boldsymbol{U}$ is a vector of real-valued functions, $\boldsymbol{H}$ is a spatial operator that contains spatial derivatives and constant coefficients, and $\boldsymbol{f}$ is a known vector of real-valued functions.

Expanding $\boldsymbol{f}(x,y,t)$ in its Taylor 's series, we can approximate the solution of system (1) by the matrix exponential (see [[1](#ref-higham2010)])

$\hat{\boldsymbol{U}}(\boldsymbol{x},t)\approx[\boldsymbol{I_p}\;0]e^{t\hat{\boldsymbol{H}}}\begin{bmatrix}\boldsymbol{U}(\boldsymbol{x},t_0)\\ \boldsymbol{e_p}\end{bmatrix},$

where $\boldsymbol{e_p}\in\mathbb{R}^p$ is the eigenvector with zero in all its entries exept the last one, that equals 1, $\hat{\boldsymbol{U}}$ is the vector function $\boldsymbol{U}$ with $p$ extra zeros at the end, $\boldsymbol{I_p}$ is the identity matrix of size $p$, and $\hat{\boldsymbol{H}}$ is the matrix

$\hat{\boldsymbol{H}}=\begin{bmatrix}\boldsymbol{H} & \boldsymbol{W}\\ 0 & \boldsymbol{J_p}\end{bmatrix}, \hspace{5.265cm}(2)$

where $\boldsymbol{J_p}$ is the zero matrix with an upper diagonal of ones

$\boldsymbol{J_p}=\begin{bmatrix}0 & 1 & 0 & 0 & \dots & 0\\ 0 & 0 & 1 & 0 & \dots & 0\\ &&\ddots&&&\vdots 
\\ 0 & 0 & 0 & 0 & \dots & 1 \\0 & 0 & 0 & 0 & \dots & 0\end{bmatrix},$

and $\boldsymbol{W}$ contains the information of the vector function $\boldsymbol{f}(\boldsymbol{x},t)$ derivatives

$\boldsymbol{W}=\begin{bmatrix}\frac{\partial^{p-1}}{\partial t^{p-1}}\boldsymbol{f}(\boldsymbol{x},t_0)\bigg\vert \frac{\partial^{p-2}}{\partial t^{p-2}}\boldsymbol{f}(\boldsymbol{x},t_0)\bigg\vert \dots \bigg\vert \frac{\partial}{\partial t}\boldsymbol{f}(\boldsymbol{x},t_0)\bigg\vert \boldsymbol{f}(\boldsymbol{x},t_0)\end{bmatrix}.$

Then, we approximate the solution operator in (2) with the m-stage Runge-Kutta (RK) method from [[2](#ref-gottlieb2003)]

\begin{align*}
        \boldsymbol{k}_0&=\boldsymbol{u}_n\\
        \boldsymbol{k}_i&=\left(\boldsymbol{I}_{n\times n}+\Delta t \hat{\boldsymbol{H}}\right)\boldsymbol{k}_{i-1},\quad i=1\dots m-1\\
        \boldsymbol{k}_m&=\sum\limits_{i=0}^{m-2}\alpha_i\boldsymbol{k}_i+\alpha_{m-1}\left(\boldsymbol{I}_{n\times n}+\Delta t \hat{\boldsymbol{H}}\right)\boldsymbol{k}_{m-1}\\
        \boldsymbol{u}_{n+1}&=\boldsymbol{k}_m,        
\end{align*}

where $\alpha_i$ are the coefficients of the Runge-Kutta and have a straightforward computation.

So, for each time step we have to construct the matrix $\hat{\boldsymbol{H}}$, where we only the submatrix $\boldsymbol{W}$ change, and apply the RK method to the vector $\boldsymbol{u}_n=[\boldsymbol{U}(\boldsymbol{x},t_n)\;\; \boldsymbol{e_p}]^T$.

So, an outline of the implementation is:

- **Environmental variables**: Set up the grid, and spatial and time variables
- **PDE System definition**: Define $\boldsymbol{f}(\boldsymbol{x},t)$ and the system of equations
- **Compute the derivatives of the source term**: symbolic computing of $\boldsymbol{f}(\boldsymbol{x},t)$ derivatives
- **Construct the operator $\hat{\boldsymbol{H}}$**: application the application of operator $\hat{\boldsymbol{H}}$
- **Implementation of the RK method**: define the required equations of the RK method to pass to Devito's operator. For this particular example, we'll use $m=3$ and $\alpha_i=1,\;\forall i\in\{0,1,2\}$.
- **Create and run Devito operator**: Executing the operator constructed with the RK equations

## 0. Import Required Libraries

First, we import all necessary libraries including NumPy for numerical operations, SymPy for symbolic mathematics, and Devito components for finite difference operations.

In [3]:
import numpy as np
import sympy as sym
from devito import (Grid, Function, TimeFunction,
                    Derivative, Operator, Eq)
from devito import configuration
from devito.symbolics import uxreplace

## 1. Environmental variables

Configure the simulation environment, including logging level, domain parameters, and computational grid setup.

In [4]:
# Configure Devito logging
configuration['log-level'] = 'DEBUG'

# Simulation parameters
extent = (1, 1)           # Physical domain size
shape = (201, 201)        # Grid resolution
origin = (0, 0)           # Domain origin

# Create computational grid
grid = Grid(origin=origin, extent=extent, shape=shape, dtype=np.float64)
x, y = grid.dimensions
t, dt = grid.time_dim, grid.stepping_dim.spacing

print(f"Grid created: {shape} points over domain {extent}")
print(f"Spatial dimensions: x={x}, y={y}")
print(f"Time dimension: t={t}, dt={dt}")

Grid created: (201, 201) points over domain (1, 1)
Spatial dimensions: x=x, y=y
Time dimension: t=time, dt=dt


## 2. PDE System definition

Create the TimeFunction objects for the wavefield variables (displacement u and velocity v) and define the source terms with both spatial and temporal characteristics.

In [5]:
# Define wavefield unknowns: u (displacement) and v (velocity)
fun_labels = ['u_multi_stage', 'v_multi_stage']
U_multi_stage = [TimeFunction(name=name, grid=grid, space_order=2, time_order=1, dtype=np.float64) for name in fun_labels]

print("Wavefield functions created:")
for i, u in enumerate(U_multi_stage):
    print(f"  {fun_labels[i]}: {u}")

# Source definition
src_spatial = Function(name="src_spat", grid=grid, space_order=2, dtype=np.float64)
src_spatial.data[100, 100] = 1  # Point source at grid center
src_temporal = sym.exp(- 100 * (t - 0.01) ** 2)  # Gaussian pulse

print(f"\nSource spatial function: {src_spatial}")
print(f"Source temporal function: {src_temporal}")

# PDE right-hand side: du/dt = v, dv/dt = ∇²u
system_eqs_rhs = [U_multi_stage[1],  # du/dt = v
                  Derivative(U_multi_stage[0], (x, 2), fd_order=2) +
                  Derivative(U_multi_stage[0], (y, 2), fd_order=2)]  # dv/dt = ∇²u

# Source coupling: [spatial, temporal, associated variable]
src = [[src_spatial, src_temporal, U_multi_stage[0]],
       [src_spatial, src_temporal * 10, U_multi_stage[0]],
       [src_spatial, src_temporal, U_multi_stage[1]]]

print(f"\nPDE system matrix H:")
for i, rhs in enumerate(system_eqs_rhs):
    print(f"  d{fun_labels[i]}/dt = {rhs}")


Allocating host memory for src_spat(205, 205) [328 KB]


Wavefield functions created:
  u_multi_stage: u_multi_stage(t, x, y)
  v_multi_stage: v_multi_stage(t, x, y)

Source spatial function: src_spat(x, y)
Source temporal function: exp(-100*(time - 0.01)**2)

PDE system matrix H:
  du_multi_stage/dt = v_multi_stage(t, x, y)
  dv_multi_stage/dt = Derivative(u_multi_stage(t, x, y), (x, 2)) + Derivative(u_multi_stage(t, x, y), (y, 2))


## 3. Compute the derivatives of the source term

Implement the core helper function that compute time derivatives of source and calculate the source derivatives up to the specified degree (deg=3).

In [6]:
def source_derivatives(deg, src, src_index, t):
    """
    Compute time derivatives of the source up to given degree.
    
    Parameters:
    -----------
    deg : int
        Degree of derivatives to compute
    src : list
        List of source terms
    src_index : list
        Indices for source terms
    t : symbol
        Time symbol
        
    Returns:
    --------
    f_deriv : list
        List of derivative expressions
    """
    f_deriv = [[src[i][1] for i in range(len(src))]]
    
    # Compute derivatives up to order p
    for _ in range(deg - 1):
        f_deriv.append([f_deriv[-1][i].diff(t) for i in range(len(src_index))])
    
    f_deriv.reverse()
    return f_deriv

print("  - source_derivatives(): Computes time derivatives of source wavelet")

# Create mapping from wavefield variables to indices
src_index_map = {val: i for i, val in enumerate(U_multi_stage)}
print("Source index mapping:")
for var, idx in src_index_map.items():
    print(f"  {var} -> {idx}")

# Extract source indices based on associated variables
src_index = [src_index_map[val] for val in [src[i][2] for i in range(len(src))]]
print(f"\nSource indices: {src_index}")

# Degree of derivatives to compute
deg = 3

# Compute source derivatives
src_deriv = source_derivatives(deg, src, src_index, t)
print(f"\nSource derivatives computed up to degree {deg}")
print(f"Number of derivative levels: {len(src_deriv)}")
print("Sample derivative expressions:")
for i, deriv in enumerate(src_deriv):
    print(f"  Level {deg-1-i}: {deriv}")

  - source_derivatives(): Computes time derivatives of source wavelet
Source index mapping:
  u_multi_stage(t, x, y) -> 0
  v_multi_stage(t, x, y) -> 1

Source indices: [0, 0, 1]

Source derivatives computed up to degree 3
Number of derivative levels: 3
Sample derivative expressions:
  Level 2: [(2.0 - 200*time)**2*exp(-100*(time - 0.01)**2) - 200*exp(-100*(time - 0.01)**2), 10*(2.0 - 200*time)**2*exp(-100*(time - 0.01)**2) - 2000*exp(-100*(time - 0.01)**2), (2.0 - 200*time)**2*exp(-100*(time - 0.01)**2) - 200*exp(-100*(time - 0.01)**2)]
  Level 1: [(2.0 - 200*time)*exp(-100*(time - 0.01)**2), 10*(2.0 - 200*time)*exp(-100*(time - 0.01)**2), (2.0 - 200*time)*exp(-100*(time - 0.01)**2)]
  Level 0: [exp(-100*(time - 0.01)**2), 10*exp(-100*(time - 0.01)**2), exp(-100*(time - 0.01)**2)]


## 4.Construct the operator $\hat{\boldsymbol{H}}$

 Application of the spatial operator to the vector formed by [u e_p]^T.

In [7]:
# and handle source term inclusion in the PDE system at each Runge-Kutta stage
def source_inclusion(rhs, u, k, src_index, src_deriv, e_p, t, dt, n_eq):
    """
    Add source terms to the PDE system at each Runge-Kutta stage.
    
    Parameters:
    -----------
    rhs : list
        Right-hand side of PDE system without sources
    u : list  
        Wavefield variables
    k : list
        Runge-Kutta stage variables
    src_index : list
        Source indices
    src_deriv : list
        Source derivatives
    e_p : list
        Expansion coefficients of the source term Taylor's series
    t : symbol
        Time symbol
    dt : symbol
        Time step symbol
    n_eq : int
        Number of equations
        
    Returns:
    --------
    src_lhs : list
        Operator application to the vector [u, e_p]^T including source terms
    e_p : list
        Updated expansion coefficients of the source term Taylor's series
    """
    # Replace wavefield variables with stage variables
    src_lhs = [uxreplace(rhs[i], {u[m]: k[m] for m in range(n_eq)}) for i in range(n_eq)]
    
    p = len(src_deriv)
    
    # Add source contributions
    for i in range(p):
        if e_p[i] != 0:
            for j in range(len(src_index)):
                src_lhs[src_index[j]] += src[j][0] * src_deriv[i][j].subs({t: t * dt}) * e_p[i]
    
    # Update expansion coefficients of the source term Taylor's series
    e_p = [e_p[i] + dt * e_p[i + 1] for i in range(p - 1)] + [e_p[-1]]
    
    return src_lhs, e_p

print("  - source_inclusion(): Apply the spatial operator of the PDE system at RK stages")

  - source_inclusion(): Apply the spatial operator of the PDE system at RK stages


## 5. Implementation of the RK method

Construct the multi-stage time integration scheme with initialization, multiple RK stages, and final update. This implements a toy example of a class of High-Order Runge-Kutta (HORK) methods, with proper source term integration.

In [8]:
n_eq = 2                  # Number of PDE unknowns (u, v)

# Temporary variables for Runge-Kutta stages
k = [Function(name=f'k{i}', grid=grid, space_order=2, time_order=1, dtype=U_multi_stage[i].dtype) for i in range(n_eq)]
# Previous stage variables needed for temporary storage
k_old = [Function(name=f'k0{i}', grid=grid, space_order=2, time_order=1, dtype=U_multi_stage[i].dtype) for i in range(n_eq)]

print(f"\nRunge-Kutta temporary variables:")
print(f"  k: {[ki.name for ki in k]}")
print(f"  k_old: {[ki.name for ki in k_old]}")

# Initialize expansion coefficients of the source term Taylor's series
e_p = [0] * deg
eta = 1
e_p[-1] = 1 / eta
print(f"\nExpansion coefficients initialized of the source term Taylor's series:")
print(f"  e_p = {e_p}")
print(f"  eta = {eta}")

# Initialize SSPRK coefficients (toy example)
alpha = [1]*4
print(f"\nSSPRK coefficients initialized:")
print(f"  alpha = {alpha}")


# Initialize list to store all stage equations
stage_eqs = []

print("Building Runge-Kutta-like scheme...")

# Stage 0: Initialization
print("  Stage 0: Initialization")
stage_eqs.extend([Eq(k[i], U_multi_stage[i]) for i in range(n_eq)])
[stage_eqs.append(Eq(U_multi_stage[i].forward, U_multi_stage[i] * alpha[0])) for i in range(n_eq)]

# Stage 1
print("  Stage 1: First RK stage")
[stage_eqs.append(Eq(k_old[j], k[j])) for j in range(n_eq)]
src_lhs, e_p = source_inclusion(system_eqs_rhs, U_multi_stage, k_old, src_index, src_deriv, e_p, t, dt, n_eq)
[stage_eqs.append(Eq(k[j], k_old[j] + dt * src_lhs[j])) for j in range(n_eq)]
[stage_eqs.append(Eq(U_multi_stage[j].forward, U_multi_stage[j].forward + k[j] * alpha[1])) for j in range(n_eq)]

# Stage 2
print("  Stage 2: Second RK stage")
[stage_eqs.append(Eq(k_old[j], k[j])) for j in range(n_eq)]
src_lhs, e_p = source_inclusion(system_eqs_rhs, U_multi_stage, k_old, src_index, src_deriv, e_p, t, dt, n_eq)
[stage_eqs.append(Eq(k[j], k_old[j] + dt * src_lhs[j])) for j in range(n_eq)]

# Stage 3 and final update
print("  Stage 3: Final RK stage and update")
[stage_eqs.append(Eq(k_old[j], k[j])) for j in range(n_eq)]
src_lhs, _ = source_inclusion(system_eqs_rhs, U_multi_stage, k_old, src_index, src_deriv, e_p, t, dt, n_eq)
[stage_eqs.append(Eq(k[j], k_old[j] + dt * src_lhs[j])) for j in range(n_eq)]
[stage_eqs.append(Eq(U_multi_stage[j].forward, U_multi_stage[j].forward + k[j] * alpha[deg - 1])) for j in range(n_eq)]

print(f"\nTotal equations created: {len(stage_eqs)}")
print("Scheme structure:")
for i, stage in enumerate(stage_eqs):
    print(f"  Stage {i}: {stage}")



Runge-Kutta temporary variables:
  k: ['k0', 'k1']
  k_old: ['k00', 'k01']

Expansion coefficients initialized of the source term Taylor's series:
  e_p = [0, 0, 1.0]
  eta = 1

SSPRK coefficients initialized:
  alpha = [1, 1, 1, 1]
Building Runge-Kutta-like scheme...
  Stage 0: Initialization
  Stage 1: First RK stage
  Stage 2: Second RK stage
  Stage 3: Final RK stage and update

Total equations created: 20
Scheme structure:
  Stage 0: Eq(k0(x, y), u_multi_stage(t, x, y))
  Stage 1: Eq(k1(x, y), v_multi_stage(t, x, y))
  Stage 2: Eq(u_multi_stage(t + dt, x, y), u_multi_stage(t, x, y))
  Stage 3: Eq(v_multi_stage(t + dt, x, y), v_multi_stage(t, x, y))
  Stage 4: Eq(k00(x, y), k0(x, y))
  Stage 5: Eq(k01(x, y), k1(x, y))
  Stage 6: Eq(k0(x, y), dt*(k01(x, y) + 11.0*src_spat(x, y)*exp(-100*(time*dt - 0.01)**2)) + k00(x, y))
  Stage 7: Eq(k1(x, y), dt*(src_spat(x, y)*exp(-100*(time*dt - 0.01)**2) + Derivative(k00(x, y), (x, 2)) + Derivative(k00(x, y), (y, 2))) + k01(x, y))
  Stage 8: E

## 6. Create and Run Devito Operator

Compile all the equations into a Devito Operator and execute the simulation with the specified parameters.

In [11]:
# Create the Devito Operator
print("Creating Devito Operator...")
op = Operator(stage_eqs, subs=grid.spacing_map)

print("Operator successfully created!")
print(f"  Number of equations: {len(stage_eqs)}")
print(f"  Grid spacing substitutions applied: {grid.spacing_map}")

# Define simulation parameters
dt_value = 0.001    # Time step size
time_max = 2000     # Maximum simulation time

print(f"\nSimulation parameters:")
print(f"  Time step (dt): {dt_value}")
print(f"  Maximum time: {time_max}")

# Execute the simulation
print("\nRunning simulation...")
op(dt=dt_value, time=time_max)

print("Simulation completed successfully!")

# Display final wavefield shapes and some statistics
print(f"\nFinal wavefield statistics:")
for i, u in enumerate(U_multi_stage):
    print(f"  {fun_labels[i]}:")
    print(f"    Shape: {u.data.shape}")
    print(f"    Max value: {np.max(u.data):.6f}")
    print(f"    Min value: {np.min(u.data):.6f}")
    print(f"    Mean value: {np.mean(u.data):.6f}")

Creating Devito Operator...


Operator `Kernel` generated in 0.33 s
  * lowering.Clusters: 0.17 s (52.7 %)
     * specializing.Clusters: 0.10 s (31.0 %)
  * lowering.IET: 0.11 s (34.1 %)
Flops reduction after symbolic optimization: [209 --> 106]
Operator `Kernel` fetched `/tmp/devito-jitcache-uid1000/ead4e1022d510052a8b9b1fb08d861635f2bdbdd.c` in 0.08 s from jit-cache
Allocating host memory for k0(205, 205) [328 KB]
Allocating host memory for k00(205, 205) [328 KB]
Allocating host memory for k01(205, 205) [328 KB]
Allocating host memory for k1(205, 205) [328 KB]
Allocating host memory for u_multi_stage(2, 205, 205) [657 KB]
Allocating host memory for v_multi_stage(2, 205, 205) [657 KB]


Operator successfully created!
  Number of equations: 20
  Grid spacing substitutions applied: {h_x: np.float64(0.005), h_y: np.float64(0.005)}

Simulation parameters:
  Time step (dt): 0.001
  Maximum time: 2000

Running simulation...


InvalidArgument: No value found for parameter time_size

## 7. References


<a id="ref-higham2010"></a>
[1] Al-Mohy AH, Higham NJ (2010) A new scaling and squaring algorithm for
the matrix exponential. SIAM Journal on Matrix Analysis and Applications
31(3):970–989

<a id="ref-gottlieb2003"></a>
[2] Gottlieb S, Gottlieb LAJ (2003) Strong stability preserving properties of runge–kutta time discretization methods for linear constant coefficient operators. Journal of Scientific Computing 18(1):83–109